In [71]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, cracked_moment_inertia
from concrete_design.steel_reinforcement import rebar_df as rebar
from masonry_design.material_properties import NW_CMU, cmu_elastic_modulus
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Concrete Slab Design
## Inputs

### Materials and Geometry

In [72]:
# Materials
fc = 3000.0 #psi
fy = 60000.0 #psi

# Dimensions
ln = 24.5 * 12 #in
b = 12.0 #in
t_slab = 10 #in
cover_b = 1 #in
cover_t = 3/4 #in

# bar spacing
n = 1 #bars per width
s = b / n
f"bar spacing = {round(s, 2)} in"

'bar spacing = 12.0 in'

In [73]:
# Provided Steel
bar = "#6"
Ab = rebar.loc[bar, "Area"]
db = rebar.loc[bar, "Diameter"]
n = 1 #number of bars per width, b

As = n * Ab #in^2
As_ft = As * 12 / b
f"As = {round(As, 3)} in^2 or {round(As_ft, 3)} in^2/ft"

'As = 0.44 in^2 or 0.44 in^2/ft'

### Loads

In [74]:
DL = conc_density * t_slab/12
LL = 40 #psf

LN = ln / 12 #ft
D=DL*b/12
L=LL*b/12
w_u = load_combos.LRFD(D, L) #plf

Mu = w_u * LN**2 / 8 * 12 #lb-in
f"Mu = {round(Mu / 12, 2)} lb-ft"

'Mu = 16056.69 lb-ft'

## Strength Design

In [75]:
# Compression Steel
bar = "#6"
Ab_c = rebar.loc[bar, "Area"]
db_c = rebar.loc[bar, "Diameter"]
n = 1 #number of bars per width, b

As_c = n * Ab_c #compression reinforcement
d_c = cover_t + db_c/2
d = t_slab - cover_b - db/2

f"d_c = {round(d_c, 3)} in; d = {round(d, 3)} in"

'd_c = 1.125 in; d = 8.625 in'

In [76]:
# Flexural Strength
include_compression_steel = "Yes"
if include_compression_steel == "No":
    As_c = 0

phi = 0.9
a = (As - As_c) * fy / (0.85 * fc * b)
Mn = As * fy * (d - a/2) + As_c * fy * (d-d_c)
phiMn = phi * Mn

print(f"Mu = {round(Mu / 12, 2)} lb-ft")
print(f"phiMn = {round(phiMn / 12, 2)} lb-ft")
phiMn > Mu

Mu = 16056.69 lb-ft
phiMn = 31927.5 lb-ft


True

## Deflection

### Short-term Deflection

In [77]:
w_a = load_combos.ASD(D, L)

Ma = w_a * LN**2 / 8 * 12 #lb-in
f"Ma = {round(Ma / 12, 2)} lb-ft"

'Ma = 12380.16 lb-ft'

In [78]:
fr = 7.5 * sqrt(fc) #psi
Ig = b * t_slab**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in")
yt = t_slab / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

Ig = 1000.0 in


'Mcr = 6846.53 lb-ft'

In [79]:
# Elastic Modulus
Ec = elastic_modulus(fc=3000) #ksi
print(f"Ec = {round(Ec, 2)} ksi")
# Moment of Inertia
Ie = Ig
Icr = cracked_moment_inertia(b=b, h=t_slab, d=d, d_c=d_c, Ec=Ec, As=As, As_c=As_c)
print(f"Icr = {round(Icr, 2)} in^4")
if Ma > 2/3*Mcr:
    Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
    print(f"Ie = {round(Ie, 2)} in")
else:
    print("section uncracked")

#Elastic Deflection
defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * 1000 * Ie)
f"Elastic Deflection = {round(defl_elastic, 3)} in"

Ec = 3320.56 ksi
Icr = 203.23 in^4
Ie = 227.91 in


'Elastic Deflection = 1.767 in'

### Long-term Deflection

In [80]:
# Deflection Factor
xi = 2.0 #duration factor
As_c = n * Ab_c #compression reinforcement
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 0.761'

In [81]:
# Long-Term Deflection
defl_long = defl_elastic * lmbda_dlt #in
f"Long-Term Deflection = {round(defl_long, 3)} in"

'Long-Term Deflection = 1.344 in'

In [82]:
# Total Deflection
defl_total = defl_elastic + defl_long
f"Total Deflection = {round(defl_total, 3)} in"

'Total Deflection = 3.112 in'

## Minimum Requirements

In [83]:
# Minimum Thickness if deflection is not calculated
t_min = ln / 30
print(f"t_min (two way) = {round(t_min, 2)} in")

t_min = ln / 20
print(f"t_min (one way) = {round(t_min, 2)} in")

t_min (two way) = 9.8 in
t_min (one way) = 14.7 in


In [84]:
# Minimum Steel
Ag = b * t_slab #in^2
As_min = 0.0018 * Ag
As_minft = As_min * 12 / b

f"As,min = {round(As_min, 3)} in^2 or {round(As_minft, 3)} in^2/ft"

'As,min = 0.216 in^2 or 0.216 in^2/ft'